# Multiple Experiments & Batch scRNAseq Analysis - ScanPy & Scanorama
Author: **Gabriel Emilio Herrera Oropeza** <br/>
Date: October 27th, 2021

### Import libraries

In [ ]:
import numpy as np
import pandas as pd
import scanpy as sc
import scanorama
import matplotlib.pyplot as plt
import os

sc.settings.verbosity = 3

### Enter path

In [ ]:
path = "/Volumes/emilio_passport/wellcome/rotations/spagnoli/data/GSE101099_RAW/"

### Read files

In [ ]:
data_e12b1 = sc.read_loom(f"{path}E12_B1/velocyto/E12_B1.loom")
data_e12b1.var_names_make_unique()

data_e12b2 = sc.read_loom(f"{path}E12_B2/velocyto/E12_B2.loom")
data_e12b2.var_names_make_unique()

data_e14b1 = sc.read_loom(f"{path}E14_B1/velocyto/E14_B1.loom")
data_e14b1.var_names_make_unique()

data_e14b2 = sc.read_loom(f"{path}E14_B2/velocyto/E14_B2.loom")
data_e14b2.var_names_make_unique()

data_e17b1 = sc.read_loom(f"{path}E17_B1/velocyto/E17_B1.loom")
data_e17b1.var_names_make_unique()

data_e17b2 = sc.read_loom(f"{path}E17_B2/velocyto/E17_B2.loom")
data_e17b2.var_names_make_unique()

In [ ]:
# Merge files but not yet integrate.
adata = data_e12b1.concatenate(data_e12b2,
                               data_e14b1,
                               data_e14b2,
                               data_e17b1,
                               data_e17b2,
                               batch_categories = [
                                   "E12_B1",
                                   "E12_B2",
                                   "E14_B1",
                                   "E14_B2",
                                   "E17_B1",
                                   "E17_B2"
                               ]
                               )

del(data_e12b1, data_e12b2, data_e14b2, data_e17b1, data_e17b2)

Summary of datasets.

In [ ]:
print(adata.obs['batch'].value_counts())

### Calculate quality control

In [ ]:
# mitochondrial genes
adata.var['mt'] = adata.var_names.str.startswith('MT-') 

# ribosomal genes
adata.var['ribo'] = adata.var_names.str.startswith(("RPS","RPL"))

# hemoglobin genes.
adata.var['hb'] = adata.var_names.str.contains(("^HB[^(P)]"))

In [ ]:
sc.pp.calculate_qc_metrics(adata, qc_vars = ['mt','ribo','hb'], 
                           percent_top = None, 
                           log1p = False, 
                           inplace = True
                          )

In [ ]:
mito_genes = adata.var_names.str.startswith('MT-')

# For each cell compute fraction of counts in mito genes vs all genes
adata.obs['percent_mt2'] = np.sum(adata[:, mito_genes].X, axis=1).A1 / np.sum(adata.X, axis=1).A1

# Add the total counts per cell as observations-annotation to adata
adata.obs['n_counts'] = adata.X.sum(axis=1).A1

Plot quality control:

In [ ]:
sc.pl.violin(adata, ['n_genes_by_counts', 'total_counts', 'pct_counts_mt','pct_counts_ribo', 'pct_counts_hb'],
             jitter=0.4, groupby = 'batch', rotation = 45)

In [ ]:
sc.pl.scatter(adata,
              x = 'total_counts',
              y = 'pct_counts_mt',
              color = "batch"
             )

### Filtering
Filter cells with low amount of reads as well as genes that are present in at least a certain amount of cells.

In [ ]:
sc.pp.filter_cells(adata, min_genes = 200)
sc.pp.filter_genes(adata, min_cells = 3)

We can also see which genes contribute the most to such reads.

In [ ]:
sc.pl.highest_expr_genes(adata, n_top = 20)

Mitochondrial genes and counts filtering.

In [ ]:
adata = adata[adata.obs.n_genes_by_counts < 6000, :]
adata = adata[adata.obs.pct_counts_mt < 0.3, :]

### Normalise and Log1 data

In [ ]:
sc.pp.normalize_total(adata, target_sum = 1e4)

In [ ]:
sc.pp.log1p(adata)

### Detect highly variable genes
Variable genes can be detected across the full dataset, but then we run the risk of getting many batch-specific genes that will drive a lot of the variation. Or we can select variable genes from each batch separately to get only celltype variation.

Detect variable genes in each dataset separately using the batch_key parameter.

In [ ]:
sc.pp.highly_variable_genes(adata, min_mean = 0.0125, max_mean = 3, n_top_genes = 5_000,
                            min_disp = 0.5, batch_key = 'batch')

print(f"Highly variable genes intersection: {sum(adata.var.highly_variable_intersection)}\n")

print("Number of batches where gene is variable:")
print(f"{adata.var.highly_variable_nbatches.value_counts()}\n")

var_genes_batch = adata.var.highly_variable_nbatches > 0

Compare overlap of the variable genes.

In [ ]:
print(f"Any batch var genes: {sum(var_genes_batch)}")
print(f"Variable genes in all batches: {sum(adata.var.highly_variable_nbatches == 6)}")

Select all genes that are variable in at least 2 datasets and use for remaining analysis.

In [ ]:
var_select = adata.var.highly_variable_nbatches > 2
var_genes = var_select.index[var_select]
len(var_genes)

### Data Integration and Correction

In [ ]:
# Split per batch into new objects.
batches = adata.obs['batch'].cat.categories.tolist()
alldata = {}
for batch in batches:
    alldata[batch] = adata[adata.obs['batch'] == batch][:,var_genes]

In [ ]:
# Convert to list of AnnData objects
adatas = list(alldata.values())

Scanorama correction and integration:

In [ ]:
corrected = scanorama.correct_scanpy(adatas, 
                                     return_dimred = True # Data is dimensionally reduced too, NO PCA required later
                                    )

In [ ]:
# Add layers to data
for n, a in enumerate(adatas):
    for l in a.layers:
        corrected[n].layers[l] = a.layers[l]

Merge adatas again.

In [ ]:
adata_corr = corrected[0].concatenate(*corrected[1:],
                                      batch_categories = [
                                          "E12_B1",
                                          "E12_B2",
                                          "E14_B1",
                                          "E14_B2",
                                          "E17_B1",
                                          "E17_B2"
                                      ]
                                     )

Save raw data (to have data of ALL genes).

In [ ]:
adata_corr.raw = adata
del(adata, corrected)

Scale data:

In [ ]:
sc.pp.scale(adata_corr, max_value = 10)

### Dimension reduction - UMAP

Calculate neighbours:

In [ ]:
sc.pp.neighbors(adata_corr, n_neighbors = 15, n_pcs = 50, 
                use_rep = "X_scanorama")

In [ ]:
sc.tl.umap(adata_corr)

Show batch location:

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
sc.pl.umap(adata_corr, color = 'batch', ax = ax, size = 20,
          alpha = 0.6)
fig.tight_layout()
fig.savefig("/Volumes/emilio_passport/wellcome/rotations/spagnoli/figures/scData/corrected/umap_batches.pdf")
plt.show()

Show marker genes:

In [ ]:
marker_genes = ["Cdh1", "Krt19", "Chga", "Vim", "Col3a1", "Pecam1", "Rac2"]

for gene in marker_genes:
    fig, ax = plt.subplots(figsize = (10,8))
    sc.pl.umap(adata_corr, 
               color = gene, 
               ax = ax, 
               size = 20,
               alpha = 0.6
              )
    fig.tight_layout()
    fig.savefig(f"/Volumes/emilio_passport/wellcome/rotations/spagnoli/figures/scData/corrected/umap_{gene}.pdf")
    plt.show()

Find clusters:

In [ ]:
sc.tl.louvain(adata_corr, resolution = 0.3)

Plot clusters:

In [ ]:
fig, ax = plt.subplots(figsize = (10,8))
sc.pl.umap(adata_corr, color = "louvain", ax = ax, size = 20,
          alpha = 0.6)
fig.tight_layout()
fig.savefig("/Volumes/emilio_passport/wellcome/rotations/spagnoli/figures/scData/corrected/louvain_clusters_identified.pdf")
plt.show()

Identify clusters based on marker genes:

In [ ]:
adata_corr.obs["louvain"].cat.categories = ['Mesenchymal', 'Epithelial', 'Mesothelial', 'Endocrine', 'Immune', 
                                            '5', '6', '7', 'Endothelial']

Measure differential expression among clusters:

In [ ]:
sc.tl.rank_genes_groups(adata_corr, 'louvain', method = 'wilcoxon')

Plot differential expression among clusters:

In [ ]:
sc.pl.rank_genes_groups(adata_corr, n_genes = 25, sharey = False, ax = ax, 
                        save = "cluster_geneComp_corr.pdf")

In [ ]:
sc.pl.rank_genes_groups_dotplot(adata_corr, n_genes=5, groupby="louvain",
                                save = "cluster_geneComp_dotplot_corr.pdf"
                               )

### Save analysed AnnData

In [ ]:
adata_corr.write('/Volumes/emilio_passport/wellcome/rotations/spagnoli/data/scData/corrected/Sneddon_corrected.h5ad')